In [1]:
# execute to import notebook styling for tables and width etc.
from IPython.core.display import HTML
import urllib.request
response = urllib.request.urlopen('https://raw.githubusercontent.com/DataScienceUWL/DS775v2/master/ds755.css')
HTML(response.read().decode("utf-8"));
import pandas as pd
import numpy as np

<font size=18>Homework 11: Recommender Systems 2</font>

# Build a Baseline Model and Compute the RMSE

The file **rating_final.csv** (found in the presentation download for this lesson) contains user ratings for overall, food, and service for various restaurants.   

Do the following:

* display the first 5 lines of the data (get familiar with the data frame)
* find the minimum restaurant rating
* find the maximum restaurant rating
* adjust the rating scale by shifting up 1 if 0 is included
* calculate the mean restaurant rating for all restaurant (just to get an idea)
* drop the ratings for food and service so that only the overall rating remains
* split the data set so that 80\% of a users ratings are in the training set and 20\% are in the testing set
* build a baseline model that assigns the appropriate rating for all predictions and compute the RMSE of these on the testing set

Click <a href = https://www.kaggle.com/uciml/restaurant-data-with-consumer-ratings> here </a> or <a href = https://archive.ics.uci.edu/ml/datasets/Restaurant+%26+consumer+data> here </a> for more details about the data set.

<font color = "blue"> *** 10 points -  answer in cells below *** (don't delete this cell) </font>

In [2]:
df = pd.read_csv("data/restaurant/rating_final.csv")
df.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [3]:
print(df.rating.min(), df.rating.max())

if df.rating.isin([0]).any():
    df['rating'] = df['rating'] + 1

0 2


In [4]:
round(df['rating'].mean(), 2)

2.2

In [5]:
df = df.drop(['food_rating','service_rating'],axis=1)

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

x = df.copy()
y = df['userID']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, stratify=y, random_state=42)

In [7]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def baseline(user_id, place_id):
    return 2

def score(cf_model, x_test):
    id_pairs = zip(x_test['userID'], x_test['placeID'])
    y_pred = np.array([cf_model(user, place) for (user, place) in id_pairs])
    y_true = np.array(x_test['rating'])
    return rmse(y_true, y_pred)

In [8]:
score(baseline, x_test)

0.7996780468043202

# Build a Weighted Mean User-Based Filter

From data in the file **rating_final.csv**, build a ratings matrix from the data frame of users, places, and restaurant ratings and build a user-based collaborative filtering model that weights mean rank using cosine similarity among users.  Fit the model on the training set and compute the RMSE for this model using the test set and compare it to the RMSE of the baseline model.  Is it better than baseline?  (*i.e.* is the RMSE smaller?)

<font color = "blue"> *** 10 points -  answer in cells below *** (don't delete this cell) </font>

In [15]:
x_train.head()

,userID,placeID,rating
830,U1089,132875,2
229,U1014,135043,2
256,U1134,135046,3
1039,U1073,132951,1
56,U1021,132740,3


In [16]:
r_matrix = x_train.pivot(values='rating', index='userID', columns='placeID')
r_matrix.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
U1002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN
U1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
U1004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
U1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
r_matrix_dummy = r_matrix.copy().fillna(0)
r_matrix_dummy.head()

placeID,132560,132561,132564,132572,132583,132584,132594,132608,132609,132613,...,135080,135081,135082,135085,135086,135088,135104,135106,135108,135109
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
U1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,0.0
U1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
U1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
U1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim.head()

userID,U1001,U1002,U1003,U1004,U1005,U1006,U1007,U1008,U1009,U1010,...,U1129,U1130,U1131,U1132,U1133,U1134,U1135,U1136,U1137,U1138
userID,,,,,,,,,,,,,,,,,,,,,
U1001,1.000000,0.053262,0.000000,0.000000,0.0,0.000000,0.073030,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.072548,0.0,0.000000,0.165145,0.000000,0.057417,0.000000
U1002,0.053262,1.000000,0.249343,0.238197,0.0,0.116952,0.116692,0.0,0.315807,0.000000,...,0.0,0.0,0.0,0.405726,0.0,0.100391,0.307860,0.333448,0.321107,0.404281
U1003,0.000000,0.249343,1.000000,0.000000,0.0,0.319801,0.000000,0.0,0.542810,0.107443,...,0.0,0.0,0.0,0.135850,0.0,0.392165,0.103081,0.115784,0.322550,0.218668
U1004,0.000000,0.238197,0.000000,1.000000,0.0,0.109109,0.163299,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.432590,0.0,0.000000,0.246183,0.155543,0.128388,0.000000
U1005,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.126491,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.238366,0.000000,0.000000,0.000000


In [19]:
def cf_user_wmean(user_id, place_id):
    
    if place_id in r_matrix:
        sim_scores = cosine_sim[user_id]
        m_ratings = r_matrix[place_id]
        idx = m_ratings[m_ratings.isnull()].index
        m_ratings = m_ratings.dropna()
        sim_scores = sim_scores.drop(idx)
        
        if sim_scores.sum()>0:
            wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
        else: 
            wmean_rating = 2
    
    else:
        wmean_rating = 2
    
    return wmean_rating

In [20]:
score(cf_user_wmean, x_test)

0.8672488443837231

# Build a kNN-Based Collaborative Filter

From data in the file **rating_final.csv**, use the *surprise* library in Python to build an kNN-based collaborative filtering model for the restaurant ratings.  Fit the model on the training set and compute the RMSE for this model on the test and compare it to the RMSEs of the baseline and weighted mean user-based models.

<font color = "blue"> *** 10 points -  answer in cells below *** (don't delete this cell) </font>

In [29]:
#Import the required classes and methods from the surprise library
from surprise import Reader, Dataset, KNNBasic
from surprise.model_selection import cross_validate
np.random.seed(123123)

reader = Reader()

data = Dataset.load_from_df(x_train, reader)

knn = KNNBasic()

output = cross_validate(knn, data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8928  0.8537  0.8695  0.7494  0.8877  0.8506  0.0525  
Fit time          0.00    0.00    0.00    0.00    0.00    0.00    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    


In [38]:
output['test_rmse'].mean()

0.8506340410064247

The RMSE for each model used so far are stated below ranked from best to worst:

- baseline model: RMSE = 0.799
- weighted-mean user-based collaborative filter: RMSE = 0.867
- kNN-based collaborative filter: (average) RMSE = 0.8506

# Build a Hybrid Filter

From data in the files **rating_final.csv** and **geoplaces2.csv** build a recommender system that is a hybrid of a metadata content-based recommender and the SVD collaborative filter.  Your recommender should do the following:

* Take in a user ID and restaurant name as user input
* Use a metadata content-based model to compute the 25 most similar restaurants based on cosine similarity from the following details (create a soup as done for the content-based recommender from Lesson 10)
    - price
    - dress code
    - accessibility
    - ambiance
    - alcohol
    - smoking area
* Compute the predicted ratings that the user might give to these 25 restaurants using the SVD collaborative filter
* Print price, dress code, accessibility, ambiance, alcohol, and smoking area to see if they are similar for the predicted restaurants.
* Return the top 10 restaurant recommendations along with their predicted ratings when user **U1077** enters the restaurant named **Restaurante Tiberius**. 
* Also return the top 10 restaurant recommendations along with their predicted ratings when user **U1065** enters the restaurant named **Restaurante Tiberius** and comment on the similarities and differences in the resulting recommendations. 

*Note 1: This data set does not have the issue of having two different ID's in separate files for each restaurant like the movie data used in the textbook example, so you won't need to use the cell for mapping ID's to titles.*

*Note 2: because of the small number of words in the "soup", many of the restaurant pairs have a cosine similarity of 1, which will affect the predicted ratings and recommendations.  This means you will have to use a different method for excluding the cosine similarity of the item with itself (use the **del** function).* 

<font color = "blue"> *** 10 points -  answer in cells below *** (don't delete this cell) </font>

In [63]:
geoplaces = pd.read_csv("data/restaurant/geoplaces2.csv")
merged_df = x.merge(geoplaces, left_on="placeID", right_on="placeID")
merged_df = merged_df.drop(['latitude','longitude','the_geom_meter','country','address','fax','zip','city','state','url','franchise','area','other_services'],axis=1)
merged_df = merged_df.rename(columns={'Rambience': 'ambience'})

In [64]:
merged_df.head()

,userID,placeID,rating,name,alcohol,smoking_area,dress_code,accessibility,price,ambience
0,U1077,135085,3,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar
1,U1108,135085,2,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar
2,U1081,135085,2,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar
3,U1056,135085,3,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar
4,U1134,135085,3,Tortas Locas Hipocampo,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,familiar


In [65]:
# drop words to lowercase and replace spaces
def sanitize(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
def create_soup(df):
    features = ['alcohol','smoking_area','dress_code','accessibility','price','ambience']
    
    for i, feature in enumerate(features):
        df[feature] = df[feature].apply(sanitize)
 
        if i == 0:
            soup_list = df[feature].copy()
        else:
            soup_list += " " + df[feature]
            
    df = df.assign(soup=soup_list)

    return(df)

In [ ]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(new_df['soup'])
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

new_df = new_df.reset_index()
indices2 = pd.Series(new_df.index, index=new_df['title'])

In [ ]:
#Build the SVD based Collaborative filter
from surprise import SVD, Reader, Dataset
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1,11))
ratings = pd.read_csv('./data/BX-Book-Ratings-3000.csv')
data = Dataset.load_from_df(ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)

#data.split(n_folds=5)
#svd = SVD()
#trainset = data.build_full_trainset()
#svd.train(trainset)

algo = SVD()
trainset = data.build_full_trainset()
cross_validate(algo,data,cv=5,verbose=True)

In [ ]:
def hybrid(user_id, isbn):
    
    # Obtain the index of the item that matches the isbn
    #idx = indices[isbn]
    
    #Extract the similarity scores and their corresponding index for every item from the cosine_sim matrix
    sim_scores = list(enumerate(cosine_sim_item[isbn]))
    
    #excluding the similarity score of the item with itself
    #del sim_scores[isbn]
    
    #Sort the (index, score) tuples in decreasing order of similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    #Select the top 25 tuples, excluding the first 
    #(as it is the similarity score of the item with itself)
    sim_scores = sim_scores[1:25]
    
    #Store the cosine_sim indices of the top 25 items in a list
    item_indices = [i[0] for i in sim_scores]

    #Extract the metadata of the aforementioned items
    items = bx.iloc[item_indices][['ISBN']]
    
    #Compute the predicted ratings using the SVD filter
    items['est_rating'] = items['ISBN'].apply(lambda x: algo.predict(user_id, x).est)
    
    #Sort the items in decreasing order of predicted rating
    items = items.sort_values('est_rating', ascending=False)
    
    #Return the top 10 items as recommendations
    return items.head(10)